In [2]:
# -*- coding: utf-8 -*-

In [3]:
import os
import pandas as pd 
from glob import glob

import sys
import time, re, pickle, itertools
import urllib3, json
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import re

import nltk
from __future__ import print_function, unicode_literals
from gensim.models.word2vec import Word2Vec
from sklearn.preprocessing import normalize as sknorm
import numpy as np
from operator import itemgetter
import html

from ckonlpy.tag import Twitter as ctwitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Twitter, Mecab,Kkma
from konlpy.utils import pprint

import chat_bot as cb
import Database_Handler as dh

C:\Users\pc\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
ct = ctwitter()
ot = Twitter()
ot = Twitter()
mecab = Mecab()
xxxx = ct._dictionary._pos2words
print (list(map(lambda x: len(xxxx[x]), xxxx)))

score_weights = {
    'num_nouns': -0.1,
    'num_words': -0.2,
    'no_noun': -1
}

def my_score(candidate):
    num_nouns = len([w for w,t in candidate if t == 'Noun'])
    num_words = len(candidate)
    no_noun = 1 if num_nouns == 0 else 0
    
    score = (num_nouns * score_weights['num_nouns'] 
             + num_words * score_weights['num_words']
             + no_noun * score_weights['no_noun'])
    return score

ct.set_selector(score_weights, my_score)

[512, 1385487, 5532]
[('이', 'Noun'), ('것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]
best: [('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]


# 자연어 처리

## TextRank 

In [5]:
stopwords = open('./data/stopwordsList.txt','r',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

참고
[인용사이트](http://excelsior-cjh.tistory.com/entry/TextRank%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EB%AC%B8%EC%84%9C%EC%9A%94%EC%95%BD)

In [6]:

class SentenceTokenizer(object):
    def __init__(self):
        self.twitter = ot
        self.ctwitter = ct
        self.mecab = mecab
        
        #self.stopwords = []
        self.stopwords = stopwords
    def text2sentences(self, text):
        sentences = re.sub('\. ','.\n', text).split('\n')
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.ctwitter.phrases(str(sentence))
                                                    if noun not in self.stopwords and len(noun) > 1]))
                nouns.append(' '.join([noun for noun in self.ctwitter.nouns(str(sentence))
                                                  if noun not in self.stopwords and len(noun) > 1]))
                nouns.append(' '.join([noun for noun in self.ctwitter.morphs(str(sentence))
                                                   if noun not in self.stopwords and len(noun) > 1]))
                #nouns.append(' '.join([noun for noun in self.mecab.nouns(str(sentence))
                #                                  if noun not in self.stopwords and len(noun) > 1]))
                #nouns.append(' '.join([noun for noun in self.mecab.phrases(str(sentence))
                #                                  if noun not in self.stopwords and len(noun) > 1]))
                #nouns.append(' '.join([noun for noun in self.mecab.morphs(str(sentence))
                #                                   if noun not in self.stopwords and len(noun) > 1]))
        #nouns = list(set(nouns))
        return nouns
    
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer(ngram_range=(1,5),sublinear_tf=True, lowercase=False,min_df=0.00001)
        self.cnt_vec = CountVectorizer(ngram_range=(1,5), lowercase=False,min_df=0.00001)
        self.graph_sentence = []
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        #tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    def build_words_graph(self, sentence):
        cnt_vec_mat = sknorm(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

class TextRankX(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        return summary
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords

# 분석

In [7]:
path = './data/Keyword_outcome/ckonlpy-daum.*'
extracted_file = glob(path)
extracted = dict()
for file in extracted_file:
    ex = pickle.load(open(file,'rb'))
    extracted.update(ex)
outlen = str(len(extracted_file))

outname = './data/Keyword_outcome/ckonlpy-daum.pickled_'+outlen

In [ ]:
import networkx
import re
 
class RawSentence:
    def __init__(self, textIter):
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s

class RawSentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s

class RawTagger:
    def __init__(self, textIter, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)

class RawTaggerReader:
    def __init__(self, filepath, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)

class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0
 
 
    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1
 
        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1
 
        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])
 
    def loadSents(self, sentenceIter, tokenizer = None):
        import math
        def similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))
 
        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            if len(s) < 2: continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)
 
        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = similarity(sentSet[i], sentSet[j])
                if s < self.threshold: continue
                self.dictBiCount[i, j] = s
 
    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
 
    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])
 
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def extract(self, ratio = 0.1):
        ranks = self.rank()
        cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        pairness = {}
        startOf = {}
        tuples = {}
        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi
 
        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            #print(k[0], l[0], pairness[k, l])
            if k not in startOf: startOf[k] = (k, l)
 
        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
 
        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if used.intersection(set(k)): continue
            both[k] = tuples[k]
            for w in k: used.add(w)
 
        #for k in cand:
        #    if k not in used or True: both[k] = ranks[k] * self.getI(k)
 
        return both

    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))

In [ ]:
extract_keywords_dict = dict()
#site = 'Naver'
site = 'daum'
if site == 'daum':
    binList = []
    collection = 'newsDaum'
    etri_outcome = '/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/data/ETRI_OUTCOME/daum2/'
elif site.lower() == 'naver':
    binList = []
    collection = 'newsNaver'
    etri_outcome = '/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/data/ETRI_OUTCOME/naver2/'

mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
useCollection = dh.Use_Collection(useDb, collection)
keywordList = list()
collection = useCollection.find({'site':site})
count = 0
for data in collection:
    idIs = data['_id']._ObjectId__id
    tis2 = html.unescape(data['title'])+'. '+html.unescape(data['mainText'])
    if data['keywords'] !='NaN':
        keywordList += list(map(lambda x: ''.join(x.split(' · ')[-1].split(' ')), data['keywords']))
    else:
        pass
    if not idIs.hex() in extracted:
        #print (data['link'])
        print (data['title'])
        print (data['keywords'])
        try:
            workrank = TextRankX(tis2)
        except:
            try:
                workrank = TextRankX(tis2)
            except:
                workrank = TextRankX(tis2)
                break
        extract_keywords_dict[idIs.hex()]= workrank.keywords()
        tr = TextRank(window=10, coef=0.0)
        print('Load...')
        stopword = list(map(lambda x: mecab.pos(x), stopwords))
        tr.load(RawTagger(tis2,mecab), lambda w: w not in stopword and (w[1] in ['VA','SN','NNG','NNB','NNP']))
        print('Build...')
        tr.build()
        kw = tr.extract(0.175)
        for k in sorted(kw, key=kw.get, reverse=True):
            print("%s\t%g" % (k, kw[k]))
        

김국진·노홍철·한혜진 '한명회', 오늘(5일) 9회만에 종영
['노홍철', '한명회', '한혜진', '종영', '편성시간']
Load...
Build...
(('이름', 'NNG'),)	0.188295
(('방송', 'NNG'), ('시간', 'NNG'))	0.141973
(('한명회', 'NNP'),)	0.127336
(('9', 'SN'),)	0.100703
(('출연자', 'NNG'),)	0.0914895
(('5', 'SN'),)	0.0884946
(('마이데일리', 'NNP'),)	0.0732738
(('녹화', 'NNG'),)	0.0729572
(('없', 'VA'),)	0.0727534
(('수', 'NNB'),)	0.0691618
(('출연', 'NNG'),)	0.0639823
(('특집', 'NNG'),)	0.0593397
'몸 만드는 게 우선' 류중일 감독, 캠프 명단에 신인 없다
['류중일']
Load...
Build...
(('류중일', 'NNP'), ('감독', 'NNG'))	0.202645
(('캠프', 'NNG'), ('명단', 'NNG'))	0.193872
(('신인', 'NNG'), ('선수', 'NNG'))	0.140367
(('1', 'SN'),)	0.0853156
(('2', 'SN'),)	0.0848091
(('일정', 'NNG'),)	0.0833035
(('있', 'VA'),)	0.0759133
(('무리', 'NNG'),)	0.070811
(('오키나와', 'NNP'),)	0.0672288
(('몸', 'NNG'),)	0.0670799
(('내년', 'NNP'),)	0.0591202
(('연습', 'NNG'),)	0.0577554
(('참가', 'NNG'),)	0.0536805
(('투수', 'NNP'),)	0.0524847
(('8', 'SN'),)	0.0522932
[어제TV]첫방 '저글러스' 男보스 女비서, 시대착오적 설정 '불편'
['불륜', '비', '첫방', '최다니엘', '최대철']
Load...
Build

Load...
Build...
(('화', 'NNP'), ('엔터테인먼트', 'NNP'))	0.238649
(('판', 'NNG'), ('청청', 'NNG'))	0.226346
(('판청', 'NNP'), ('청', 'NNP'))	0.188706
(('판빙빙', 'NNP'),)	0.106584
(('위', 'NNG'),)	0.101659
(('연습생', 'NNG'),)	0.0898335
(('남동생', 'NNG'),)	0.0887676
(('중국', 'NNP'),)	0.0879394
이문세 "자이언티 신곡 '눈' 충격..해석 위해 200번 들어"
['자이언티', '발라드곡', '음원차트', '피처링', '엠넷']
Load...
Build...
(('겨울', 'NNG'), ('노래', 'NNG'))	0.162108
(('등', 'NNB'), ('국내', 'NNG'))	0.147719
(('이번', 'NNG'), ('신곡', 'NNG'))	0.138568
(('자이언티', 'NNP'),)	0.135052
(('이문세', 'NNP'),)	0.116785
(('곡', 'NNG'),)	0.0932069
(('수', 'NNB'),)	0.0719305
(('지니뮤직', 'NNP'),)	0.0644849
(('눈', 'NNG'),)	0.0626172
(('음악', 'NNG'),)	0.059651
(('애정', 'NNG'),)	0.0580621
(('후배', 'NNG'),)	0.0566749
(('사랑', 'NNG'),)	0.0521269
[N톡] "박세리가 왜?" 美LPGA, '정글' 제작진에 연락한 이유
['박세리', '닐슨코리아', '김병만', '리얼', '승부욕']
Load...
Build...
(('뉴스', 'NNG'), ('1', 'SN'))	0.138862
(('박세리', 'NNP'),)	0.118423
(('예능', 'NNG'), ('프로그램', 'NNG'))	0.108339
(('처음', 'NNG'), ('방송', 'NNG'))	0.102615
(('정글', 

Load...
Build...
(('전', 'NNG'), ('원장', 'NNG'))	0.142912
(('스탠퍼드대', 'NNP'), ('아시아', 'NNP'))	0.112591
(('국정원', 'NNG'),)	0.104811
(('수사', 'NNG'), ('팀', 'NNG'))	0.102651
(('객원', 'NNG'), ('연구원', 'NNG'))	0.100332
(('대학', 'NNG'), ('기부', 'NNG'))	0.0923805
(('등', 'NNB'), ('자료', 'NNG'))	0.083683
(('태평양', 'NNP'), ('연구소', 'NNG'))	0.0800199
(('것', 'NNB'),)	0.0763016
(('차장', 'NNG'),)	0.0549427
(('200', 'SN'),)	0.0533315
(('이자', 'NNG'),)	0.0525048
(('국가안보전략연구원', 'NNP'),)	0.0523971
(('혐의', 'NNG'),)	0.0454981
(('민간', 'NNG'),)	0.0453045
(('댓글부대', 'NNP'),)	0.0441248
(('지급', 'NNG'),)	0.0436985
(('원', 'NNG'),)	0.0433199
(('후', 'NNG'),)	0.0432727
(('최근', 'NNG'),)	0.0410425
(('같', 'VA'),)	0.0409936
[SPO 이슈] '성실한 쑥(SUK)', 현지 코멘트 "한국의 즐라탄" 칭찬(영상)
NaN
Load...
Build...
(('리그', 'NNG'), ('앙', 'NNG'))	0.121945
(('시즌', 'NNG'), ('트루아', 'NNP'))	0.103112
(('석현준', 'NNP'),)	0.100122
(('멀티', 'NNG'), ('골', 'NNG'))	0.0916358
(('한국', 'NNP'), ('시간', 'NNG'))	0.0851049
(('모나코', 'NNP'), ('전', 'NNG'))	0.0848941
(('성공', 'NNG'),)	0

Load...
Build...
(('부장', 'NNG'), ('판사', 'NNG'))	0.176184
(('부심', 'NNG'), ('결과', 'NNG'))	0.137505
(('구속', 'NNG'),)	0.103145
(('수', 'NNB'), ('없', 'VA'))	0.0990964
(('국방부', 'NNG'), ('정책실장', 'NNP'))	0.0807503
(('김명수', 'NNP'), ('대법원장', 'NNG'))	0.0804337
(('법원', 'NNG'),)	0.0656679
(('법관', 'NNG'),)	0.0613224
(('비판', 'NNG'),)	0.0581935
(('혐의', 'NNG'),)	0.0580137
(('법치주의', 'NNG'),)	0.0524869
(('정치', 'NNG'),)	0.0509289
(('판결', 'NNG'),)	0.0505859
(('임관빈', 'NNP'),)	0.0499222
(('납득', 'NNG'),)	0.0470646
(('것', 'NNB'),)	0.0465726
(('글', 'NNG'),)	0.0437306
(('김관진', 'NNP'),)	0.0436219
(('위선', 'NNG'),)	0.0422785
(('김', 'NNP'),)	0.03986
[최고의 TV] "추측만 무성"..'황금빛' 민부장 서경화, 정체가 뭐야
['서지수', '노명희', '유인영', '신혜선', '나영희']
Load...
Build...
(('민부', 'NNG'), ('장', 'NNG'))	0.222619
(('내', 'NNB'), ('인생', 'NNG'))	0.176319
(('분', 'NNP'),)	0.134317
(('해성그룹', 'NNP'),)	0.0968065
(('서지수', 'NNP'),)	0.0789674
(('집', 'NNG'),)	0.0733061
(('황금빛', 'NNP'),)	0.0721371
(('시청자', 'NNG'),)	0.0711451
(('추측', 'NNG'),)	0.0700313
(('드라마', 'N

Load...
Build...
(('외국인', 'NNG'), ('선수', 'NNG'))	0.109054
(('6', 'SN'), ('패', 'NNG'))	0.105123
(('계약', 'NNG'), ('협상', 'NNG'))	0.103775
(('시즌', 'NNG'), ('도중', 'NNG'))	0.0921058
(('33', 'SN'), ('홀드', 'NNG'))	0.0891525
(('허프', 'NNP'), ('영입', 'NNG'))	0.0845609
(('올해', 'NNG'), ('야쿠르트', 'NNP'))	0.079317
(('평균자책점', 'NNP'), ('2', 'SN'))	0.0778588
(('일본', 'NNP'), ('구단', 'NNG'))	0.0667982
(('루키', 'NNP'),)	0.0652703
(('등', 'NNB'),)	0.062388
(('뷰캐넌', 'NNP'),)	0.0611279
(('연봉', 'NNG'),)	0.0589409
(('좋', 'VA'),)	0.0490477
(('투수', 'NNP'),)	0.0484303
(('팀', 'NNG'),)	0.0461987
(('선발', 'NNG'),)	0.0458049
(('훈련', 'NNG'),)	0.0442951
(('것', 'NNB'),)	0.0428062
(('경기', 'NNG'),)	0.0421019
(('관심', 'NNG'),)	0.0358999
물음표의 한화 선발진, 똑똑한 '넘버3'는 나올까
NaN
Load...
Build...
(('외국인', 'NNG'), ('투수', 'NNG'))	0.123033
(('패', 'NNG'), ('평균', 'NNG'))	0.116281
(('토종', 'NNG'), ('선발', 'NNG'))	0.114469
(('자책', 'NNG'), ('4', 'SN'))	0.0998184
(('한용덕', 'NNP'), ('감독', 'NNG'))	0.0980993
(('시즌', 'NNG'), ('10', 'SN'))	0.0849939
(('자원', '

...

Load...
Build...
(('대표', 'NNG'), ('팀', 'NNG'))	0.166064
(('김신욱', 'NNP'),)	0.118534
(('일본', 'NNP'), ('도쿄', 'NNP'))	0.111188
(('3', 'SN'), ('골', 'NNG'))	0.0941784
(('역할', 'NNG'),)	0.0845377
(('감독', 'NNG'),)	0.0843778
(('전북', 'NNP'),)	0.0718454
(('것', 'NNB'),)	0.0641555
(('아시아', 'NNP'),)	0.0593167
(('자리', 'NNG'),)	0.0591598
(('후반', 'NNG'),)	0.0588717
(('경기', 'NNG'),)	0.0572556
(('힘들', 'VA'),)	0.0570519
(('플레이', 'NNG'),)	0.0569119
(('리그', 'NNG'),)	0.0563545
(('1', 'SN'),)	0.0531234
(('볼', 'NNG'),)	0.0523132
[맨시티-토트넘] 손흥민, 맨시티전 평점 6.3 .. 더 브라위너 10점 만점
['손흥민', '토트넘 핫스퍼 FC', '케빈 데 브루잉']
Load...
Build...
(('후반', 'NNG'), ('18', 'SN'))	0.182706
(('평점', 'NNG'), ('6', 'SN'))	0.155049
(('맨시티', 'NNP'),)	0.115828
(('손흥민', 'NNP'),)	0.104819
(('맨체스터', 'NNP'),)	0.0901473
(('토트넘', 'NNP'),)	0.0755351
(('1', 'SN'),)	0.0705304
(('케인', 'NNP'),)	0.0687171
(('한국', 'NNP'),)	0.0679926
(('스타디움', 'NNG'),)	0.0677754
(('스코', 'NNP'),)	0.0673005
이혜천, 호주 리그에서 여전히 땀흘리는 베테랑
NaN
Load...
Build...
(('야구', 'NNG'), ('리그', 'NN

Build...
(('샤이니', 'NNP'), ('종현', 'NNP'))	0.3112
(('채', 'NNB'), ('발견', 'NNG'))	0.222808
(('오후', 'NNG'), ('청담동', 'NNP'))	0.164384
(('사망', 'NNG'),)	0.0913499
(('김', 'NNP'),)	0.0909386
(('18', 'SN'),)	0.0873296
(('누리', 'NNG'),)	0.0827379
(('충격', 'NNG'),)	0.0781477
(('소속사', 'NNP'),)	0.0715548
'우리반 애가 괴롭혀' 편지 품고 투신한 초등생.. 수사 착수
['초등생', '성동구', '초등학교']
Load...
Build...
(('반', 'NNG'), ('학생', 'NNG'))	0.210546
(('초등', 'NNG'), ('학교', 'NNG'))	0.163297
(('군', 'NNB'),)	0.109405
(('군', 'NNG'),)	0.102083
(('지난달', 'NNG'),)	0.0888426
(('투신', 'NNG'),)	0.0795574
(('등', 'NNB'),)	0.0767343
(('경찰', 'NNG'),)	0.0738813
(('성동구', 'NNP'),)	0.0690882
(('혐의', 'NNG'),)	0.0648867
(('목숨', 'NNG'),)	0.0625728
황병서, 별 떼이고 6계급 '강등'..모든 직위 박탈
['황병서', '별', '검열']
Load...
Build...
(('황병서', 'NNP'), ('총정치국장', 'NNP'))	0.162414
(('6', 'SN'), ('계급', 'NNG'))	0.148604
(('지난달', 'NNG'), ('국회', 'NNG'))	0.127084
(('처벌', 'NNG'),)	0.112992
(('것', 'NNB'),)	0.104806
(('북한', 'NNP'), ('권력', 'NNG'))	0.103421
(('강등', 'NNG'),)	0.090718
(('최룡해', 'N

Load...
Build...
(('기초', 'NNG'), ('연금', 'NNG'))	0.202114
(('노인', 'NNG'), ('단독', 'NNG'))	0.188593
(('선정', 'NNG'), ('기준액', 'NNP'))	0.180801
(('소득', 'NNG'),)	0.0886685
(('수급자', 'NNP'),)	0.0828129
(('가구', 'NNG'),)	0.0788202
(('금액', 'NNG'),)	0.0774044
(('재산', 'NNG'),)	0.0762485
(('최대', 'NNG'),)	0.0731905
(('정액', 'NNG'),)	0.069077
(('초과', 'NNG'),)	0.067711
(('이하', 'NNG'),)	0.0573736
주담대 금리 年4.6%까지 껑충.. 1억 대출때 15만원 더 낸다
['금리', '변동금리', '대출금리']
Load...
Build...
(('주택담보대출', 'NNP'), ('금리', 'NNG'))	0.266436
(('은행', 'NNG'),)	0.14684
(('70', 'SN'),)	0.13686
(('대출', 'NNG'),)	0.110856
(('4', 'SN'),)	0.106376
"이 나이에 또 공부해야할 줄이야.." 생존위해 책과 씨름하는 5060
['공부', '일자리', '수험생']
Load...
Build...
(('준비', 'NNG'), ('중', 'NNB'))	0.166596
(('자격증', 'NNG'),)	0.114583
(('김', 'NNP'), ('모', 'NNG'))	0.105992
(('5060', 'SN'), ('수험', 'NNG'))	0.0939136
(('도서관', 'NNG'), ('4', 'SN'))	0.0926704
(('데', 'NNB'), ('일자리', 'NNG'))	0.0896751
(('시험', 'NNG'), ('공부', 'NNG'))	0.0823324
(('책', 'NNG'),)	0.0733939
(('재취업', 'NNG'),)	0.0701567


Load...
Build...
(('아베', 'NNP'), ('총리', 'NNG'))	0.307176
(('중국', 'NNP'),)	0.118547
(('대표', 'NNG'),)	0.118421
(('일본', 'NNP'),)	0.117137
(('외교', 'NNG'),)	0.105074
(('의자', 'NNG'),)	0.101602
안철수 "文 방중은 낙제점, 洪, 아베 알현외교는 창피"
['이슈 · 문 대통령 중국 국빈 방문', '안철수', '방중', '아베신조']
Load...
Build...
(('문', 'NNG'), ('대통령', 'NNG'))	0.193781
(('알현', 'NNG'), ('외교', 'NNG'))	0.173758
(('홍', 'NNG'), ('대표', 'NNG'))	0.1728
(('아베', 'NNP'),)	0.092824
(('이야기', 'NNG'),)	0.0730907
(('방일', 'NNG'),)	0.0726227
(('비난', 'NNG'),)	0.0724128
(('방중', 'NNG'),)	0.0678486
(('중국', 'NNP'),)	0.0670667
(('합의', 'NNG'),)	0.0658535
(('안철수', 'NNP'),)	0.0629752
노인 소득 130만원·부부 208만원 안되면 기초연금 받는다
['기초연금', '근로소득', '65세']
Load...
Build...
(('기초', 'NNG'), ('연금', 'NNG'))	0.200361
(('가구', 'NNG'), ('기준', 'NNG'))	0.155581
(('선정', 'NNG'), ('기준액', 'NNP'))	0.147711
(('내년', 'NNG'), ('1', 'SN'))	0.138452
(('근로소득', 'NNP'), ('공제액', 'NNG'))	0.130821
(('소득', 'NNG'),)	0.092155
(('정액', 'NNG'),)	0.0758515
(('지급', 'NNG'),)	0.0707889
(('인상', 'NNG'),)	0.0646815
(

Load...
Build...
(('그룹', 'NNP'), ('샤이니', 'NNP'))	0.17972
(('등', 'NNB'), ('극단', 'NNG'))	0.142382
(('오후', 'NNG'), ('서울', 'NNP'))	0.142055
(('종현', 'NNP'),)	0.126654
(('발견', 'NNG'),)	0.0949899
(('것', 'NNB'),)	0.0863525
(('레지던스', 'NNP'),)	0.0832629
(('경찰', 'NNG'),)	0.0788142
(('아이돌', 'NNP'),)	0.0764928
(('이날', 'NNG'),)	0.0613513
(('병원', 'NNG'),)	0.0601314
(('휴대', 'NNG'),)	0.0571707
故 샤이니 종현, 마지막이 된 솔로 콘서트 '팬들 충격'
['샤이니종현', '콘서트', '아이돌', '슬기', '웬디']
Load...
Build...
(('샤이니', 'NNP'), ('종현', 'NNP'))	0.256372
(('솔로', 'NNG'), ('콘서트', 'NNG'))	0.213829
(('채', 'NNB'), ('발견', 'NNG'))	0.165923
(('서울', 'NNP'),)	0.0986062
(('아이돌', 'NNP'),)	0.0849976
(('박아름', 'NNP'),)	0.0716489
(('팬', 'NNG'),)	0.0688675
(('충격', 'NNG'),)	0.0613916
(('공연', 'NNG'),)	0.0612471
샤이니 종현 숨진채 발견, 마지막 SNS글 "네가 아파하지않길 기도해"
['샤이니종현', 'sm엔터테인먼트', '청담동', '스트리밍', '서울강남구']
